# Join data files

In [4]:
import pandas as pd

# List of CSV file paths
csv_files = ['ia-data_06-23_08-23.csv', 'ia-data_09-23_12-23.csv', 'ia-data_01-24_03-24.csv', 'ia-data_04-24_06-24.csv', 'ia-data_07-24_11-24.csv']

# Output file
output_file = 'raw_ia_data.csv'

# Create the output file with the header from the first CSV
with open(output_file, 'w') as f_out:
    pd.read_csv(csv_files[0], skip_blank_lines=True).to_csv(f_out, index=False, lineterminator='\n')

# Append the remaining files without the header
for file in csv_files[1:]:
    with open(output_file, 'a') as f_out:
        pd.read_csv(file, skip_blank_lines=True).to_csv(f_out, index=False, header=False, lineterminator='\n')

print("CSV files appended successfully into 'merged.csv'")


C:\Users\Dario\AppData\Local\Temp\ipykernel_9632\2468897074.py:11: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(csv_files[0], skip_blank_lines=True).to_csv(f_out, index=False, lineterminator='\n')
C:\Users\Dario\AppData\Local\Temp\ipykernel_9632\2468897074.py:16: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(file, skip_blank_lines=True).to_csv(f_out, index=False, header=False, lineterminator='\n')
C:\Users\Dario\AppData\Local\Temp\ipykernel_9632\2468897074.py:16: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(file, skip_blank_lines=True).to_csv(f_out, index=False, header=False, lineterminator='\n')
C:\Users\Dario\AppData\Local\Temp\ipykernel_9632\2468897074.py:16: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(file, sk

CSV files appended successfully into 'merged.csv'


In [2]:
import pandas as pd
df = pd.read_csv('raw_ia_data.csv')
df['Time'] = pd.to_datetime(df['Time'])
df.sort_values(by=['deviceId', 'Time'], inplace=True)
df.to_csv('raw_ia_data.csv', index=False)

C:\Users\Dario\AppData\Local\Temp\ipykernel_14356\3081564111.py:2: DtypeWarning: Columns (2,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('raw_ia_data.csv')


# Get packets previous to a failure

In [2]:
import pandas as pd
from tqdm import tqdm

ALARM_PUMP = 3
ALARM_EQUIPMENT = 5

df = pd.read_csv('raw_ia_data.csv')
df['Time'] = pd.to_datetime(df['Time'])
df.sort_values(by=['deviceId', 'Time'], inplace=True)
print('START', df.shape[0])

rows_to_drop = []  # Indices to drop
last_row = None
for index in tqdm(df.iloc[::-1].index):  # Reverse order
    current_state = df.loc[index, 'state']
    current_time = df.loc[index, 'Time']

    if current_state == ALARM_PUMP or current_state == ALARM_EQUIPMENT:    # Apply condition
        if last_row is not None:
            rows_to_drop.append(last_row['Index'])  # Mark the last row for deletion
        last_row = {'Time': current_time, 'State': current_state, 'Index': index}
    elif last_row is not None:
        time_diff = last_row['Time'] - current_time
        if  time_diff.total_seconds() > 6 * 3600:   # Less than 6 hours
          last_row = None
    else:
      rows_to_drop.append(index)  # Mark the row for deletion

# Drop rows in a single operation (more efficient)
df.drop(rows_to_drop, inplace=True)
# Reset the index after dropping rows (optional)
df.reset_index(drop=True, inplace=True)

df.to_csv('ia_data_events.csv', index=False)

df.describe()

C:\Users\Dario\AppData\Local\Temp\ipykernel_15252\2444059446.py:7: DtypeWarning: Columns (2,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('raw_ia_data.csv')


START 3162739


100%|██████████| 3162739/3162739 [01:40<00:00, 31617.75it/s]


,Time,angle,distanceSpeed,flow,latitude,longitude,pressure,rawSpeed,speed,state
count,80533,80533.000000,80533.000000,80533.000000,80533.000000,80533.000000,80533.000000,80533.000000,80533.000000,80533.000000
mean,2024-01-27 20:12:35.534936064,173.920395,22.730902,207.389628,-31.972362,-64.706832,1.267521,0.014087,0.104917,4.852744
min,2023-06-02 09:59:13,0.000000,-47.000000,43.200000,-40.740237,-112.048533,0.000000,-141.000000,0.000000,0.000000
25%,2023-10-31 06:30:21,81.800000,0.126000,120.000000,-37.621616,-65.797910,0.300000,-0.028000,0.009640,6.000000
50%,2024-01-12 13:41:26,171.000000,0.557000,181.000000,-34.085091,-62.548989,1.200000,0.003610,0.064800,6.000000
75%,2024-02-27 13:37:18,265.000000,0.983000,250.000000,-31.958102,-60.600987,1.900000,0.079600,0.136000,6.000000
max,2024-11-20 20:14:42,360.000000,488031.000000,900.000000,31.029565,-0.087667,10.000000,11.600000,141.000000,8.000000
std,NaN,104.934566,2725.175748,109.725799,11.104041,8.577342,1.075326,0.552844,0.543072,2.095278


# Get random periods with NO failures

In [ ]:
import numpy as np

